In [1]:
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.support.select import Select
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd
import time
import os
import tabula
import re
import PyPDF2
from word2numberi18n import w2n
import locale
import io
import glob
import sys
from PIL import Image
import pytesseract
from pdf2image import convert_from_path
import gc



In [2]:
os.environ['w2n.lang'] = 'es'
locale.setlocale(locale.LC_TIME, 'es_ES')

'es_ES'

In [3]:
def fill_text_field(driver, xpath, text):
    # Funcion que llena texto en un campo de la pagina
    text_input = driver.find_element(By.XPATH, xpath)
    text_input.clear()
    text_input.send_keys(text)

def select_option(driver, xpath, index):
    # Selecciona una opcion en un menu desplegable
    select_object = Select(driver.find_element(By.XPATH, xpath))
    select_object.select_by_value(index)


In [4]:
url = 'https://www.pjud.cl/tribunales/corte-suprema'
chromeOptions = webdriver.ChromeOptions()
path = os.path.join(os.getcwd(), "output2\\")
prefs = {"download.default_directory" : path,  "directory_upgrade": True}
chromeOptions.add_experimental_option("prefs",prefs)


In [5]:
driver = webdriver.Chrome(executable_path=ChromeDriverManager().install(), options = chromeOptions)
driver.get(url)
#NOTA: al abrirse el driver de Chrome, seleccionar Configuración-Descargar PDFs

[WDM] - Downloading: 100%|██████████| 6.68M/6.68M [00:00<00:00, 25.6MB/s]
C:\Users\danny\AppData\Local\Temp\ipykernel_7156\2038824681.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(executable_path=ChromeDriverManager().install(), options = chromeOptions)


In [6]:
driver.implicitly_wait(10)

In [7]:
Integraciones = driver.find_element(By.XPATH, 
"/html/body/section/div/div/div[1]/div[2]/div/div/div/div/div/div/div/div[1]/div[2]/div[1]/div/div/div[3]/div/div/a[6]")
Integraciones.click()

In [8]:
años = [2022]
meses = ['01']
url_downloads = []
#meses = ["01", "02", "03", "04", "05", "06", "07", "08", "09", "10", "11", "12"]

for año in años:
    # Estos indices se obtienen de las opciones de la pagina, cada uno es un mes, cambiar para obtener mas meses
    #for index in range(1, 2):
        # Se envia la info a la pagina
    select_option(driver, '/html/body/div[11]/div/div/div[2]/form/div[2]/div[1]/div/select', str(año))
    for mes in meses:
        select_option(driver, '/html/body/div[11]/div/div/div[2]/form/div[2]/div[2]/div/select', mes)
        select = Select(driver.find_element(By.XPATH, "/html/body/div[11]/div/div/div[2]/form/div[2]/div[3]/div/select")) #get all the options into a list
        optionsList = [item.get_attribute("value") for item in select.options]

        for optionValue in optionsList:
            select_option(driver, '/html/body/div[11]/div/div/div[2]/form/div[2]/div[3]/div/select', optionValue)
            driver.find_element(By.XPATH, '/html/body/div[11]/div/div/div[2]/form/div[2]/div[4]/button').click()
            wait = WebDriverWait(driver, 10) 
            for i in range(1,5):
                try:
                    boton_descarga = wait.until(EC.visibility_of_element_located((By.XPATH, f"/html/body/div[11]/div/div/div[2]/table/tbody/tr[{i}]/td[1]/a")))
                    url_downloads.append(boton_descarga.get_attribute('href'))
                except: 
                    pass
driver.close


<bound method WebDriver.close of <selenium.webdriver.chrome.webdriver.WebDriver (session="f0e12354c86998b691c31a1e5fa9d8f5")>>

In [9]:
url_downloads

['https://www.pjud.cl/integrations/docsdownload/32087',
 'https://www.pjud.cl/integrations/docsdownload/32088',
 'https://www.pjud.cl/integrations/docsdownload/32095',
 'https://www.pjud.cl/integrations/docsdownload/32096',
 'https://www.pjud.cl/integrations/docsdownload/32087',
 'https://www.pjud.cl/integrations/docsdownload/32088',
 'https://www.pjud.cl/integrations/docsdownload/32095',
 'https://www.pjud.cl/integrations/docsdownload/32096',
 'https://www.pjud.cl/integrations/docsdownload/32098',
 'https://www.pjud.cl/integrations/docsdownload/32099',
 'https://www.pjud.cl/integrations/docsdownload/32100',
 'https://www.pjud.cl/integrations/docsdownload/32101',
 'https://www.pjud.cl/integrations/docsdownload/31840',
 'https://www.pjud.cl/integrations/docsdownload/31839',
 'https://www.pjud.cl/integrations/docsdownload/31838',
 'https://www.pjud.cl/integrations/docsdownload/32102',
 'https://www.pjud.cl/integrations/docsdownload/32104',
 'https://www.pjud.cl/integrations/docsdownload/

In [15]:
import urllib.request
for url_file in url_downloads:
    urllib.request.urlretrieve(url_file, f"Scraper/output/acta_{''.join(filter(str.isdigit, url_file))}.pdf")

In [16]:
columns = ['Fecha', 'Sala', 'Integrantes', 'Causa']
df = pd.DataFrame(columns = columns)

In [18]:
palabras = ['licencia', 'comisi ón', 'permiso', 'feriado', 'inhabilidad', 'subroga', 'vacancia']

In [33]:



pdf_files = glob.glob(os.path.join(os.getcwd(), "Scraper/output",'*.pdf'))

count = 0
extractedtext = ""
for pdf_file in pdf_files:
    print(pdf_file)
    pdfFileObj = open(pdf_file,'rb')               
    pdfReader = PyPDF2.PdfFileReader(pdfFileObj)   
    num_pages = pdfReader.numPages
    pdfPageObj = pdfReader.getPage(0)
    text = pdfPageObj.extractText()
    Causa = []
    if re.search("INSTALACI", text):
        Integrantes = re.findall('(?<=ÑOR).*?(?=\s)', text)
        try:
            Fecha = re.findall('(?<=FIRMADIGITAL).*?(?=\\\n)', text)[0]
        except:
            pass
        try:
            Sala = re.findall('(?<=\\s).*?(?=\\sSALA)', text)[0]
        except:
            pass
        for palabra in palabras:         
            if palabra in text:
                Causa.append(palabra)
        df = df.append(
               {"Fecha" : str(Fecha), "Sala" : str(Sala), "Integrantes" : Integrantes, 'Causa' : Causa},
               ignore_index=True,
            )


c:\Users\danny\Documents\web_scraping\WebScraping-Cs\Scraper/output\acta_31385.pdf
c:\Users\danny\Documents\web_scraping\WebScraping-Cs\Scraper/output\acta_31386.pdf
c:\Users\danny\Documents\web_scraping\WebScraping-Cs\Scraper/output\acta_31387.pdf
c:\Users\danny\Documents\web_scraping\WebScraping-Cs\Scraper/output\acta_31388.pdf
c:\Users\danny\Documents\web_scraping\WebScraping-Cs\Scraper/output\acta_31523.pdf
c:\Users\danny\Documents\web_scraping\WebScraping-Cs\Scraper/output\acta_31524.pdf
c:\Users\danny\Documents\web_scraping\WebScraping-Cs\Scraper/output\acta_31525.pdf
c:\Users\danny\Documents\web_scraping\WebScraping-Cs\Scraper/output\acta_31526.pdf
c:\Users\danny\Documents\web_scraping\WebScraping-Cs\Scraper/output\acta_31527.pdf


C:\Users\danny\AppData\Local\Temp\ipykernel_7156\1040401876.py:26: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(
C:\Users\danny\AppData\Local\Temp\ipykernel_7156\1040401876.py:26: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(
C:\Users\danny\AppData\Local\Temp\ipykernel_7156\1040401876.py:26: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(
C:\Users\danny\AppData\Local\Temp\ipykernel_7156\1040401876.py:26: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(
C:\Users\danny\AppData\Local\Temp\ipykernel_7156\1040401876.py:26: FutureWarning: The frame.append method is deprecated and will

c:\Users\danny\Documents\web_scraping\WebScraping-Cs\Scraper/output\acta_31528.pdf
c:\Users\danny\Documents\web_scraping\WebScraping-Cs\Scraper/output\acta_31529.pdf
c:\Users\danny\Documents\web_scraping\WebScraping-Cs\Scraper/output\acta_31530.pdf
c:\Users\danny\Documents\web_scraping\WebScraping-Cs\Scraper/output\acta_31598.pdf
c:\Users\danny\Documents\web_scraping\WebScraping-Cs\Scraper/output\acta_31599.pdf
c:\Users\danny\Documents\web_scraping\WebScraping-Cs\Scraper/output\acta_31600.pdf
c:\Users\danny\Documents\web_scraping\WebScraping-Cs\Scraper/output\acta_31601.pdf


C:\Users\danny\AppData\Local\Temp\ipykernel_7156\1040401876.py:26: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(
C:\Users\danny\AppData\Local\Temp\ipykernel_7156\1040401876.py:26: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(
C:\Users\danny\AppData\Local\Temp\ipykernel_7156\1040401876.py:26: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(
C:\Users\danny\AppData\Local\Temp\ipykernel_7156\1040401876.py:26: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(
C:\Users\danny\AppData\Local\Temp\ipykernel_7156\1040401876.py:26: FutureWarning: The frame.append method is deprecated and will

c:\Users\danny\Documents\web_scraping\WebScraping-Cs\Scraper/output\acta_31838.pdf
c:\Users\danny\Documents\web_scraping\WebScraping-Cs\Scraper/output\acta_31839.pdf
c:\Users\danny\Documents\web_scraping\WebScraping-Cs\Scraper/output\acta_31840.pdf
c:\Users\danny\Documents\web_scraping\WebScraping-Cs\Scraper/output\acta_32078.pdf
c:\Users\danny\Documents\web_scraping\WebScraping-Cs\Scraper/output\acta_32079.pdf
c:\Users\danny\Documents\web_scraping\WebScraping-Cs\Scraper/output\acta_32081.pdf
c:\Users\danny\Documents\web_scraping\WebScraping-Cs\Scraper/output\acta_32082.pdf
c:\Users\danny\Documents\web_scraping\WebScraping-Cs\Scraper/output\acta_32087.pdf
c:\Users\danny\Documents\web_scraping\WebScraping-Cs\Scraper/output\acta_32088.pdf


...

C:\Users\danny\AppData\Local\Temp\ipykernel_7156\1040401876.py:26: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(
C:\Users\danny\AppData\Local\Temp\ipykernel_7156\1040401876.py:26: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(
C:\Users\danny\AppData\Local\Temp\ipykernel_7156\1040401876.py:26: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(
C:\Users\danny\AppData\Local\Temp\ipykernel_7156\1040401876.py:26: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(
C:\Users\danny\AppData\Local\Temp\ipykernel_7156\1040401876.py:26: FutureWarning: The frame.append method is deprecated and will

c:\Users\danny\Documents\web_scraping\WebScraping-Cs\Scraper/output\acta_32109.pdf
c:\Users\danny\Documents\web_scraping\WebScraping-Cs\Scraper/output\acta_32110.pdf
c:\Users\danny\Documents\web_scraping\WebScraping-Cs\Scraper/output\acta_32169.pdf
c:\Users\danny\Documents\web_scraping\WebScraping-Cs\Scraper/output\acta_32172.pdf
c:\Users\danny\Documents\web_scraping\WebScraping-Cs\Scraper/output\acta_32173.pdf
c:\Users\danny\Documents\web_scraping\WebScraping-Cs\Scraper/output\acta_32174.pdf
c:\Users\danny\Documents\web_scraping\WebScraping-Cs\Scraper/output\acta_32236.pdf
c:\Users\danny\Documents\web_scraping\WebScraping-Cs\Scraper/output\acta_32237.pdf
c:\Users\danny\Documents\web_scraping\WebScraping-Cs\Scraper/output\acta_32239.pdf
c:\Users\danny\Documents\web_scraping\WebScraping-Cs\Scraper/output\acta_32240.pdf
c:\Users\danny\Documents\web_scraping\WebScraping-Cs\Scraper/output\acta_32347.pdf


C:\Users\danny\AppData\Local\Temp\ipykernel_7156\1040401876.py:26: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(
C:\Users\danny\AppData\Local\Temp\ipykernel_7156\1040401876.py:26: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(
C:\Users\danny\AppData\Local\Temp\ipykernel_7156\1040401876.py:26: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(
C:\Users\danny\AppData\Local\Temp\ipykernel_7156\1040401876.py:26: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(
C:\Users\danny\AppData\Local\Temp\ipykernel_7156\1040401876.py:26: FutureWarning: The frame.append method is deprecated and will

c:\Users\danny\Documents\web_scraping\WebScraping-Cs\Scraper/output\acta_32350.pdf
c:\Users\danny\Documents\web_scraping\WebScraping-Cs\Scraper/output\acta_32353.pdf
c:\Users\danny\Documents\web_scraping\WebScraping-Cs\Scraper/output\acta_32354.pdf
c:\Users\danny\Documents\web_scraping\WebScraping-Cs\Scraper/output\acta_32466.pdf
c:\Users\danny\Documents\web_scraping\WebScraping-Cs\Scraper/output\acta_32467.pdf
c:\Users\danny\Documents\web_scraping\WebScraping-Cs\Scraper/output\acta_32468.pdf
c:\Users\danny\Documents\web_scraping\WebScraping-Cs\Scraper/output\acta_32469.pdf
c:\Users\danny\Documents\web_scraping\WebScraping-Cs\Scraper/output\acta_32622.pdf
c:\Users\danny\Documents\web_scraping\WebScraping-Cs\Scraper/output\acta_32623.pdf
c:\Users\danny\Documents\web_scraping\WebScraping-Cs\Scraper/output\acta_32624.pdf
c:\Users\danny\Documents\web_scraping\WebScraping-Cs\Scraper/output\acta_32625.pdf
c:\Users\danny\Documents\web_scraping\WebScraping-Cs\Scraper/output\acta_32628.pdf
c:\U

C:\Users\danny\AppData\Local\Temp\ipykernel_7156\1040401876.py:26: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(
C:\Users\danny\AppData\Local\Temp\ipykernel_7156\1040401876.py:26: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(
C:\Users\danny\AppData\Local\Temp\ipykernel_7156\1040401876.py:26: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(
C:\Users\danny\AppData\Local\Temp\ipykernel_7156\1040401876.py:26: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(
C:\Users\danny\AppData\Local\Temp\ipykernel_7156\1040401876.py:26: FutureWarning: The frame.append method is deprecated and will

In [34]:
df['Causa'].head(50)

0                        [comisi ón, feriado]
1                                   [feriado]
2               [comisi ón, feriado, subroga]
3               [comisi ón, permiso, feriado]
4                        [comisi ón, feriado]
5                          [permiso, feriado]
6                        [comisi ón, feriado]
7      [comisi ón, permiso, feriado, subroga]
8                        [comisi ón, feriado]
9                        [comisi ón, feriado]
10                       [comisi ón, feriado]
11              [comisi ón, feriado, subroga]
12                                [comisi ón]
13                                  [feriado]
14                       [comisi ón, feriado]
15              [comisi ón, feriado, subroga]
16                                [comisi ón]
17                       [comisi ón, feriado]
18             [licencia, comisi ón, feriado]
19              [comisi ón, feriado, subroga]
20              [comisi ón, permiso, feriado]
21                                

In [35]:
#Para separar año y día y convertir a número - lleva a formato fecha. Ojo: muy largo y pesado de ejecutar
df['Fecha'] = df['Fecha'].str.replace("treinta y uno", "treintaiuno")
df['Dia'] = df['Fecha'].str.split(' ').str[0] 
df["Ano"] = df["Fecha"].str.extract(r'(\w+)$')
df['Dia'] = df['Dia'].str.replace("treintaiuno", "treinta y uno")

In [36]:
#df['Dia'] = df.Dia.apply(w2n.word_to_num)
df['Dia']

0           cuatro
1           cuatro
2           cuatro
3           cuatro
4            cinco
          ...     
131    veintisiete
132     veintiocho
133     veintiocho
134     veintiocho
135     veintiocho
Name: Dia, Length: 136, dtype: object

In [37]:
nums = {'^uno$': '1', '^dos$': '2', '^tres$': '3', "^cuatro$":'4', "^cinco$" : '5', "^seis$":'6', "^siete$": '7', 
        '^ocho$':'8',
       '^nueve$':'9', 'diez':'10', 'once':'11', 'doce':'12', 'trece':'13','catorce':'14', 'quince': '15', 'dieciséis':'16',
        'diecisiete':'17', 'dieciocho':'18', 'diecinueve':'19', 'veinte':'20', 'veintiuno':'21','veintidós':'22', 
        'veintitrés':'23', 'veinticuatro':'24', 'veinticinco':'25', 'veintiséis':'26', 'veintisiete':'27',
        'veintiocho':'28', 'veintinueve': '29', '^treinta$': '30', 'treinta y uno': '31'}
        

for old, new in nums.items():
    df['Dia'] = df['Dia'].str.replace(old, new, regex=True)

In [38]:
df['Dia'].value_counts()

4     8
5     8
27    8
24    8
21    8
20    8
19    8
18    8
28    8
13    8
11    8
10    8
17    8
7     8
6     8
12    6
25    6
14    4
Name: Dia, dtype: int64

In [40]:
nums = {'^veinte$': '20', '^veintiuno$': '21', '^veintidós$': '22'}
        

for old, new in nums.items():
    df['Ano'] = df['Ano'].str.replace(old, new, regex=True)

In [41]:
df['Ano'].value_counts()

22    136
Name: Ano, dtype: int64

In [42]:
#convierte a string para juntar
df['Ano'] = df['Ano'].astype(str)
df['Dia'] = df['Dia'].astype(str)#aisla y convierte a una columna mes
df["Mes"] = df["Fecha"]
df['Mes'] = df['Mes'].str.split(n=1).str[1]
df['Mes'] = df['Mes'].str.lstrip()
df['Mes'] = df['Mes'].str.split(' ').str[1] 

In [43]:
#concatenación fecha en formato
df['Fecha2'] = df['Dia'] + '/' + df['Mes'] + '/' + df['Ano']
#transformación a datetime
df['Fecha2'] = pd.to_datetime(df["Fecha2"], format = '%d/%B/%y')

In [44]:
df2 = df
df2['Presidente'] = df2['Integrantes'].str[0]
df2['Integrante1'] = df2['Integrantes'].str[1]
df2['Integrante2'] = df2['Integrantes'].str[2]
df2['Integrante3'] = df2['Integrantes'].str[3]
df2['Integrante4'] = df2['Integrantes'].str[4]

In [47]:
df3 = df2
df3 = df3.drop(['Fecha', 'Integrantes', 'Dia', 'Ano', 'Mes'], axis = 1)
df3

,Sala,Causa,Fecha2,Presidente,Integrante1,Integrante2,Integrante3,Integrante4
0,CUARTA,"[comisi ón, feriado]",2022-01-04,AMUÑOZ,MATUS,CONTRERAS,MUNITA,ACOPPO
1,TERCERA,[feriado],2022-01-04,MUÑOZ,CARROZA,VAZQUEZ,ALCALDE,HUMERES
2,SEGUNDA,"[comisi ón, feriado, subroga]",2022-01-04,LLANOS,ALETELIER,BIEL,MERA,ATAVOLARI
3,PRIMERA,"[comisi ón, permiso, feriado]",2022-01-04,AREPETTO,MUÑOZ,GOMEZ,AGAJARDO,FUENTES
4,CUARTA,"[comisi ón, feriado]",2022-01-05,AMUÑOZ,MERA,CONTRERAS,FUENTES,AETCHEBERRY
...,...,...,...,...,...,...,...,...
131,CUARTA,[feriado],2022-01-27,BLANCO,A,AGAJARDO,CONTRERAS,RUZ
132,PRIMERA,"[permiso, feriado]",2022-01-28,SILVA,,SILVA,MUÑOZ,GOMEZ
133,SEGUNDA,"[permiso, feriado]",2022-01-28,DAHM,MERA,VASQUEZ,A,AETCHEBERRY
134,TERCERA,[permiso],2022-01-28,MUÑOZ,AVIVANCO,ARAVANALES,,CARROZA


In [48]:
#df3['Causas'] = df3['Causa'].str[0:7] #ojo que se va a llamar Causa

df3['Causa1'] = df3['Causa'].str[0]
df3['Causa2'] = df3['Causa'].str[1]
df3['Causa3'] = df3['Causa'].str[2]
df3['Causa4'] = df3['Causa'].str[3]

In [49]:

df3 = df3.drop(['Causa'], axis = 1)

In [50]:
df3.drop_duplicates()

,Sala,Fecha2,Presidente,Integrante1,Integrante2,Integrante3,Integrante4,Causa1,Causa2,Causa3,Causa4
0,CUARTA,2022-01-04,AMUÑOZ,MATUS,CONTRERAS,MUNITA,ACOPPO,comisi ón,feriado,NaN,NaN
1,TERCERA,2022-01-04,MUÑOZ,CARROZA,VAZQUEZ,ALCALDE,HUMERES,feriado,NaN,NaN,NaN
2,SEGUNDA,2022-01-04,LLANOS,ALETELIER,BIEL,MERA,ATAVOLARI,comisi ón,feriado,subroga,NaN
3,PRIMERA,2022-01-04,AREPETTO,MUÑOZ,GOMEZ,AGAJARDO,FUENTES,comisi ón,permiso,feriado,NaN
4,CUARTA,2022-01-05,AMUÑOZ,MERA,CONTRERAS,FUENTES,AETCHEBERRY,comisi ón,feriado,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
63,CUARTA,2022-01-27,BLANCO,A,AGAJARDO,CONTRERAS,RUZ,feriado,NaN,NaN,NaN
64,PRIMERA,2022-01-28,SILVA,,SILVA,MUÑOZ,GOMEZ,permiso,feriado,NaN,NaN
65,SEGUNDA,2022-01-28,DAHM,MERA,VASQUEZ,A,AETCHEBERRY,permiso,feriado,NaN,NaN
66,TERCERA,2022-01-28,MUÑOZ,AVIVANCO,ARAVANALES,,CARROZA,permiso,NaN,NaN,NaN


In [51]:
df3.to_csv("Segundo intento")

In [32]:
url = 'https://www.pjud.cl/tribunales/corte-suprema'
chromeOptions = webdriver.ChromeOptions()
path = os.path.join(os.getcwd(), "output2\\")
prefs = {"download.default_directory" : path,  "directory_upgrade": True}
chromeOptions.add_experimental_option("prefs",prefs)
driver = webdriver.Chrome(executable_path=ChromeDriverManager().install(), options = chromeOptions)
driver.get(url)
driver.implicitly_wait(10)

[WDM] - Downloading:  50%|████▉     | 3.09M/6.21M [00:00<00:00, 31.8MB/s]<ipython-input-32-dc77de3c10e2>:6: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(executable_path=ChromeDriverManager().install(), options = chromeOptions)
[WDM] - Downloading: 100%|██████████| 6.21M/6.21M [00:20<00:00, 31.8MB/s]

In [36]:
#años = list(range(2020, 2022))
años = [2015]
meses = ["04"]

for año in años:
    # Estos indices se obtienen de las opciones de la pagina, cada uno es un mes, cambiar para obtener mas meses
    #for index in range(1, 2):
        # Se envia la info a la pagina
    select_option(driver, '/html/body/div[11]/div/div/div[2]/form/div[2]/div[1]/div/select', str(año))
    for mes in meses:
        select_option(driver, '/html/body/div[11]/div/div/div[2]/form/div[2]/div[2]/div/select', mes)
        select = Select(driver.find_element(By.XPATH, "/html/body/div[11]/div/div/div[2]/form/div[2]/div[3]/div/select")) #get all the options into a list
        optionsList = []
        for item in select.options:
            optionsList.append(item.get_attribute("value"))            
            for optionValue in optionsList:
                select_option(driver, '/html/body/div[11]/div/div/div[2]/form/div[2]/div[3]/div/select', optionValue)

        # Se hace click con los datos llenados
                driver.find_element(By.XPATH, '/html/body/div[11]/div/div/div[2]/form/div[2]/div[4]/button').click()
        
        # Se descargan los datos
                wait = WebDriverWait(driver, 10) 
    #descarga por salas cambia último tr: 1, 2, 3, 4 - no siempre está disponible, el mismo día, las mismas salas
                try: 
                    boton_descarga = wait.until(EC.visibility_of_element_located((By.XPATH, '/html/body/div[11]/div/div/div[2]/table/tbody/tr[1]/td[1]/a')))
                    boton_descarga.click()
                except: 
                    pass
                try: 
                    boton_descarga = wait.until(EC.visibility_of_element_located((By.XPATH, '/html/body/div[11]/div/div/div[2]/table/tbody/tr[2]/td[1]/a')))
                    boton_descarga.click()
                except: 
                    pass
                try: 
                    boton_descarga = wait.until(EC.visibility_of_element_located((By.XPATH, '/html/body/div[11]/div/div/div[2]/table/tbody/tr[3]/td[1]/a')))
                    boton_descarga.click()
                except: 
                    pass
                try: 
                    boton_descarga = wait.until(EC.visibility_of_element_located((By.XPATH, '/html/body/div[11]/div/div/div[2]/table/tbody/tr[4]/td[1]/a')))
                    boton_descarga.click()
                except: 
                    pass
driver.close

<bound method WebDriver.close of <selenium.webdriver.chrome.webdriver.WebDriver (session="c960b56523f7123dd5d4425ba472de68")>>

In [56]:

pytesseract.pytesseract.tesseract_cmd = "C:/Program Files (x86)/Tesseract-OCR/tesseract.exe"

In [590]:
pdf_files = glob.glob(os.path.join("C:/Users/jwile/WebScraping/scan",'*.pdf'))


columns = ['Fecha', 'Sala', 'Presidente', 'Ministros', 'Integrantes']
df = pd.DataFrame(columns = columns)
pages = []


for pdf_file in pdf_files:
    print(pdf_file)
    imgs = convert_from_path(pdf_file, 300)
    Ministros = []
    Integrantes = []
    for page in imgs:
        page.save('out.jpg', 'JPEG')
        text = pytesseract.image_to_string(page, lang ='spa')
        if re.search("INSTALACION", text):
            Presidente = re.findall('(?<=PRESIDENTE:).*?(?=\\\n)', text)
            Ministros.append(re.findall('(?<=MINISTROS:).*?(?=\\\n)', text))
            Ministros.append(re.findall('.*?(?=\sy\sEL\sABOGADO)', text))
            Integrantes.append(re.findall('(?<=INTEGRANTES:).*?(?=\\\n)', text))
            Integrantes.append(re.findall('(?<=INTEGRANTE:).*?(?=\\\n)', text))
            if re.findall('(?<=En Santiago,).*?(?=,)', text) != []:
                Fecha = re.findall('(?<=En Santiago,).*?(?=,)', text) 
            elif re.findall('(?<=En Sanfiago,).*?(?=,)', text) != []:
                Fecha = re.findall('(?<=En Sanfiago,).*?(?=,)', text) 
            else: Fecha = []
            try:
                Sala = re.findall('(?<=\\s).*?(?=\\sSALA)', text)[0]
            except:
                pass
            df = df.append(
               {"Fecha" : str(Fecha), "Sala" : str(Sala), 'Presidente' : Presidente, 'Ministros' : Ministros,
                "Integrantes" : Integrantes},
               ignore_index=True,
            )
        
        
        



C:/Users/jwile/WebScraping/scan\Sala+1 (1).pdf
C:/Users/jwile/WebScraping/scan\Sala+1 (10).pdf
C:/Users/jwile/WebScraping/scan\Sala+1 (100).pdf
C:/Users/jwile/WebScraping/scan\Sala+1 (11).pdf
C:/Users/jwile/WebScraping/scan\Sala+1 (12).pdf
C:/Users/jwile/WebScraping/scan\Sala+1 (13).pdf
C:/Users/jwile/WebScraping/scan\Sala+1 (14).pdf
C:/Users/jwile/WebScraping/scan\Sala+1 (15).pdf
C:/Users/jwile/WebScraping/scan\Sala+1 (16).pdf
C:/Users/jwile/WebScraping/scan\Sala+1 (17).pdf
C:/Users/jwile/WebScraping/scan\Sala+1 (18).pdf
C:/Users/jwile/WebScraping/scan\Sala+1 (19).pdf
C:/Users/jwile/WebScraping/scan\Sala+1 (2).pdf
C:/Users/jwile/WebScraping/scan\Sala+1 (20).pdf
C:/Users/jwile/WebScraping/scan\Sala+1 (21).pdf
C:/Users/jwile/WebScraping/scan\Sala+1 (22).pdf
C:/Users/jwile/WebScraping/scan\Sala+1 (23).pdf
C:/Users/jwile/WebScraping/scan\Sala+1 (24).pdf
C:/Users/jwile/WebScraping/scan\Sala+1 (25).pdf
C:/Users/jwile/WebScraping/scan\Sala+1 (26).pdf
C:/Users/jwile/WebScraping/scan\Sala+1 (2

C:/Users/jwile/WebScraping/scan\Sala+2 (55).pdf
C:/Users/jwile/WebScraping/scan\Sala+2 (56).pdf
C:/Users/jwile/WebScraping/scan\Sala+2 (57).pdf
C:/Users/jwile/WebScraping/scan\Sala+2 (58).pdf
C:/Users/jwile/WebScraping/scan\Sala+2 (59).pdf
C:/Users/jwile/WebScraping/scan\Sala+2 (6).pdf
C:/Users/jwile/WebScraping/scan\Sala+2 (60).pdf
C:/Users/jwile/WebScraping/scan\Sala+2 (61).pdf
C:/Users/jwile/WebScraping/scan\Sala+2 (62).pdf
C:/Users/jwile/WebScraping/scan\Sala+2 (63).pdf
C:/Users/jwile/WebScraping/scan\Sala+2 (64).pdf
C:/Users/jwile/WebScraping/scan\Sala+2 (65).pdf
C:/Users/jwile/WebScraping/scan\Sala+2 (66).pdf
C:/Users/jwile/WebScraping/scan\Sala+2 (67).pdf
C:/Users/jwile/WebScraping/scan\Sala+2 (68).pdf
C:/Users/jwile/WebScraping/scan\Sala+2 (69).pdf
C:/Users/jwile/WebScraping/scan\Sala+2 (7).pdf
C:/Users/jwile/WebScraping/scan\Sala+2 (70).pdf
C:/Users/jwile/WebScraping/scan\Sala+2 (71).pdf
C:/Users/jwile/WebScraping/scan\Sala+2 (72).pdf
C:/Users/jwile/WebScraping/scan\Sala+2 (73

C:/Users/jwile/WebScraping/scan\Sala+3 (22).pdf
C:/Users/jwile/WebScraping/scan\Sala+3 (23).pdf
C:/Users/jwile/WebScraping/scan\Sala+3 (24).pdf
C:/Users/jwile/WebScraping/scan\Sala+3 (25).pdf
C:/Users/jwile/WebScraping/scan\Sala+3 (26).pdf
C:/Users/jwile/WebScraping/scan\Sala+3 (27).pdf
C:/Users/jwile/WebScraping/scan\Sala+3 (28).pdf
C:/Users/jwile/WebScraping/scan\Sala+3 (29).pdf
C:/Users/jwile/WebScraping/scan\Sala+3 (3).pdf
C:/Users/jwile/WebScraping/scan\Sala+3 (30).pdf
C:/Users/jwile/WebScraping/scan\Sala+3 (31).pdf
C:/Users/jwile/WebScraping/scan\Sala+3 (32).pdf
C:/Users/jwile/WebScraping/scan\Sala+3 (33).pdf
C:/Users/jwile/WebScraping/scan\Sala+3 (34).pdf
C:/Users/jwile/WebScraping/scan\Sala+3 (35).pdf
C:/Users/jwile/WebScraping/scan\Sala+3 (36).pdf
C:/Users/jwile/WebScraping/scan\Sala+3 (37).pdf
C:/Users/jwile/WebScraping/scan\Sala+3 (38).pdf
C:/Users/jwile/WebScraping/scan\Sala+3 (39).pdf
C:/Users/jwile/WebScraping/scan\Sala+3 (4).pdf
C:/Users/jwile/WebScraping/scan\Sala+3 (40

C:/Users/jwile/WebScraping/scan\Sala+3 - 2022-08-22T143506.763.pdf
C:/Users/jwile/WebScraping/scan\Sala+3 - 2022-08-22T143511.242.pdf
C:/Users/jwile/WebScraping/scan\Sala+3 - 2022-08-22T143515.507.pdf
C:/Users/jwile/WebScraping/scan\Sala+3 - 2022-08-22T143518.990.pdf
C:/Users/jwile/WebScraping/scan\Sala+3 - 2022-08-22T143521.484.pdf
C:/Users/jwile/WebScraping/scan\Sala+3 - 2022-08-22T143525.355.pdf
C:/Users/jwile/WebScraping/scan\Sala+3 - 2022-08-22T143600.444.pdf
C:/Users/jwile/WebScraping/scan\Sala+3 - 2022-08-22T143603.789.pdf
C:/Users/jwile/WebScraping/scan\Sala+3 - 2022-08-22T143605.957.pdf
C:/Users/jwile/WebScraping/scan\Sala+3 - 2022-08-22T143609.042.pdf
C:/Users/jwile/WebScraping/scan\Sala+3 - 2022-08-22T143642.980.pdf
C:/Users/jwile/WebScraping/scan\Sala+3 - 2022-08-22T143643.736.pdf
C:/Users/jwile/WebScraping/scan\Sala+3 - 2022-08-22T143647.537.pdf
C:/Users/jwile/WebScraping/scan\Sala+3 - 2022-08-22T143722.971.pdf
C:/Users/jwile/WebScraping/scan\Sala+3 - 2022-08-22T143725.289

C:/Users/jwile/WebScraping/scan\Sala+4 - 2022-08-22T143034.498.pdf
C:/Users/jwile/WebScraping/scan\Sala+4 - 2022-08-22T143041.217.pdf
C:/Users/jwile/WebScraping/scan\Sala+4 - 2022-08-22T143045.946.pdf
C:/Users/jwile/WebScraping/scan\Sala+4 - 2022-08-22T143122.936.pdf
C:/Users/jwile/WebScraping/scan\Sala+4 - 2022-08-22T143126.490.pdf
C:/Users/jwile/WebScraping/scan\Sala+4 - 2022-08-22T143131.402.pdf
C:/Users/jwile/WebScraping/scan\Sala+4 - 2022-08-22T143208.885.pdf
C:/Users/jwile/WebScraping/scan\Sala+4 - 2022-08-22T143212.589.pdf
C:/Users/jwile/WebScraping/scan\Sala+4 - 2022-08-22T143215.955.pdf
C:/Users/jwile/WebScraping/scan\Sala+4 - 2022-08-22T143221.975.pdf
C:/Users/jwile/WebScraping/scan\Sala+4 - 2022-08-22T143226.550.pdf
C:/Users/jwile/WebScraping/scan\Sala+4 - 2022-08-22T143229.748.pdf
C:/Users/jwile/WebScraping/scan\Sala+4 - 2022-08-22T143233.566.pdf
C:/Users/jwile/WebScraping/scan\Sala+4 - 2022-08-22T143237.388.pdf
C:/Users/jwile/WebScraping/scan\Sala+4 - 2022-08-22T143241.939

In [651]:
df
df2 = df.copy()


In [770]:
df3 = df2.copy()

In [771]:
sum(df3['Fecha'].isna())

0

In [772]:
df3['Fecha']
df3['Fecha'].value_counts()

                                                84
[' a dieciséis de abril de dos mil quince       52
[' a veintitrés de abril de dos mil quince      43
[' a trece de abril de dos mil quince           40
[' a veintiuno de abril de dos mil quince       36
[' a diez de abril de dos mil quince            28
[' a diecisiete de abril de dos mil quince      25
[' a veintidos de abril de dos mil quince       22
[' a catorce de abril de dos mil quince         22
[' a quince de abril de dos mil quinee          18
[' catorce de abril de dos mil quince           16
[' a veinticuatro de abril de dos mil quince    15
[' a quinee de abril de dos mil quince          15
[' a quince de abril de dos mil quince          15
[' a treinta de abril de dos mil quince         14
[' veintitrés de abril de dos mil quince        13
[' a veintisiete de abril de dos mil quince     12
[' dieciséis de abril de dos mil quince         12
[' a veinfisiete de abril de dos mil quince     11
[' a veíntisiete de abril de do

In [773]:
df3['Fecha'] = df3['Fecha'].str[2:]

In [654]:
#df3['Fecha'] = df3['Fecha'].str[:-2]

In [774]:
df3['Fecha'].value_counts()

                                              84
 a dieciséis de abril de dos mil quince       52
 a veintitrés de abril de dos mil quince      43
 a trece de abril de dos mil quince           40
 a veintiuno de abril de dos mil quince       36
 a diez de abril de dos mil quince            28
 a diecisiete de abril de dos mil quince      25
 a catorce de abril de dos mil quince         22
 a veintidos de abril de dos mil quince       22
 a quince de abril de dos mil quinee          18
 catorce de abril de dos mil quince           16
 a quince de abril de dos mil quince          15
 a veinticuatro de abril de dos mil quince    15
 a quinee de abril de dos mil quince          15
 a treinta de abril de dos mil quince         14
 veintitrés de abril de dos mil quince        13
 dieciséis de abril de dos mil quince         12
 a veintisiete de abril de dos mil quince     12
 a veinfisiete de abril de dos mil quince     11
 a veíntisiete de abril de dos mil quince     11
 a veintidós de abri

In [775]:
df3['Fecha'] = df3['Fecha'].str.replace("treinta y uno", "treintaiuno")
df3['Fecha'] = df3['Fecha'].str.replace('\ba\b','')

df3['Fecha'] =  df3['Fecha'].str.replace(r"\b[a-zA-Z]\b", "")
df3['Fecha'] = df3['Fecha'].str.lstrip()
df3['Fecha']




<ipython-input-775-71d413033bce>:4: FutureWarning: The default value of regex will change from True to False in a future version.
  df3['Fecha'] =  df3['Fecha'].str.replace(r"\b[a-zA-Z]\b", "")


0             trece de abril de dos mil quince
1         dieciséis de abril de dos mil quince
2               dos de abril de dos mil quince
3           catorce de abril de dos mil quince
4         dieciséis de abril de dos mil quince
                        ...                   
687    veinticuatro de abril de dos mil quince
688            diez de abril de dos mil quince
689      diecisiete de abril de dos mil quince
690      diecisiete de abril de dos mil quince
691            diez de abril de dos mil quince
Name: Fecha, Length: 692, dtype: object

In [776]:
df3['Dia'] = df3['Fecha'].str.split().str.get(0)
df3["Ano"] = df3["Fecha"].str.extract(r'(\w+)$')
df3['Dia'] = df3['Dia'].str.replace("treintaiuno", "treinta y uno")

In [777]:
nums = {'^uno$': '1', '^dos$': '2', '^tres$': '3', "^cuatro$":'4', "^cinco$" : '5', "^seis$":'6', "^siete$": '7', 
        '^ocho$':'8',
       '^nueve$':'9', '^mueve$':'9', 'diez':'10', 'once':'11', 'doce':'12', 'trece':'13','catorce':'14', 'quince': '15', 'quinee': '15',
        'dieciséis':'16',
        'diecisiete':'17', 'dieciocho':'18', 'diecinueve':'19', 'veinte':'20', 'veintiuno':'21','veintidós':'22', 
        'veintidos':'22', 'veintifiete' : '27', 'veinfisiete': '27', 'veíntisiete' : '27',
        'veintitrés':'23', 'veinticuatro':'24', 'veinticinco':'25', 'veintiséis':'26', 'veintisiete':'27',
        'veintiocho':'28', 'veinfiocho': '28','veintinueve': '29', 'veinfinueve': '29', '^treinta$': '30', 'tremía' : '30',
        'treinta y uno': '31'}
        

for old, new in nums.items():
    df3['Dia'] = df3['Dia'].str.replace(old, new, regex=True)

In [778]:
df3['Dia'].value_counts()

16    64
23    56
15    56
13    45
21    44
22    40
27    40
14    38
10    28
29    27
30    26
17    25
9     22
8     20
7     18
28    16
24    15
4      9
2      9
6      7
1      3
Name: Dia, dtype: int64

In [779]:
nums = {'^veinte$': '20', '^veintiuno$': '21', '^veintidós$': '22', '^quince$' : '15', 'quinee' :'15', 'nan': '15',
       'quínce': '15', 'quimce':'15', 'quínee' : '15'}
        

for old, new in nums.items():
    df3['Ano'] = df3['Ano'].str.replace(old, new, regex=True)

In [780]:
df3['Ano'].value_counts()

15    608
Name: Ano, dtype: int64

In [781]:
#convierte a string para juntar
df3['Ano'] = df3['Ano'].astype(str)
df3['Dia'] = df3['Dia'].astype(str)#aisla y convierte a una columna mes
df3["Mes"] = df3["Fecha"]
df3['Mes'] = df3['Mes'].str.split(n=1).str[1]
df3['Mes'] = df3['Mes'].str.lstrip()
df3['Mes'] = df3['Mes'].str.split(' ').str[1] 

In [782]:
df3['Mes'].value_counts()
meses = {'abrij': 'abil', 'abríl': 'abril', 'de': 'abril', 'abil' : 'abril'}
        

for old, new in meses.items():
    df3['Mes'] = df3['Mes'].str.replace(old, new, regex=True)

df3['Mes'].value_counts()

abril    608
Name: Mes, dtype: int64

In [783]:
#concatenación fecha en formato
df3['Fecha2'] = df3['Dia'] + '/' + df3['Mes'] + '/' + df3['Ano']
#transformación a datetime
df3['Fecha2'] = pd.to_datetime(df3["Fecha2"], format = '%d/%B/%y')

In [784]:
df3

,Fecha,Sala,Presidente,Ministros,Integrantes,Dia,Ano,Mes,Fecha2
0,trece de abril de dos mil quince,PRIMERA,[ SEÑOR SEGURA.],"[[ SEÑORES VALDES, SILVA seÑora MAGGI y], []]","[[], []]",13,15,abril,2015-04-13
1,dieciséis de abril de dos mil quince,PRIMERA,[ SEÑOR VALDÉS.],"[[ SEÑORA MA GGI, señor FUENTES y LOS ABOGADOS...","[[ SEÑORES CORREA y FIGUEROA.], []]",16,15,abril,2015-04-16
2,dos de abril de dos mil quince,PRIMERA,[ SEÑOR VALDES.],"[[ SEÑORES SILVA señorsa MAGGI, sEÑor], [FUENT...","[[], [ SEÑOR GÓMEZ.]]",2,15,abril,2015-04-02
3,catorce de abril de dos mil quince,PRIMERA,[ SEÑOR SEGURA.],"[[ SEÑORES VALDÉS, SILVA señora MAGGI y EL], []]","[[], [ SEÑOR FIGUEROA.]]",14,15,abril,2015-04-14
4,dieciséis de abril de dos mil quince,PRIMERA,[ SEÑOR VALDÉS.],"[[ SEÑORA MA GGI, señor FUENTES y LOS ABOGADOS...","[[ SEÑORES CORREA y FIGUEROA.], []]",16,15,abril,2015-04-16
...,...,...,...,...,...,...,...,...,...
687,veinticuatro de abril de dos mil quince,CUARTA,[ SEÑOR MUÑOZ.],"[[ SEÑORES DOLMESTCH, CARREÑO, PIERRY,], []]","[[], []]",24,15,abril,2015-04-24
688,diez de abril de dos mil quince,CUARTA,[ SEÑOR MUÑOZ.],"[[ SEÑORES JUICA, DOLMESTCH, VALDES,], []]","[[], []]",10,15,abril,2015-04-10
689,diecisiete de abril de dos mil quince,CUARTA,[ SEÑOR MUÑOZ.],"[[ SEÑORES JUICA, DOLMESTCH, VALDES,], []]","[[], []]",17,15,abril,2015-04-17
690,diecisiete de abril de dos mil quince,CUARTA,[ SEÑOR MUÑOZ.],"[[ SEÑORES JUICA, DOLMESTCH, VALDES,], []]","[[], []]",17,15,abril,2015-04-17


In [785]:
df4 = df3.copy()

In [786]:
df4

,Fecha,Sala,Presidente,Ministros,Integrantes,Dia,Ano,Mes,Fecha2
0,trece de abril de dos mil quince,PRIMERA,[ SEÑOR SEGURA.],"[[ SEÑORES VALDES, SILVA seÑora MAGGI y], []]","[[], []]",13,15,abril,2015-04-13
1,dieciséis de abril de dos mil quince,PRIMERA,[ SEÑOR VALDÉS.],"[[ SEÑORA MA GGI, señor FUENTES y LOS ABOGADOS...","[[ SEÑORES CORREA y FIGUEROA.], []]",16,15,abril,2015-04-16
2,dos de abril de dos mil quince,PRIMERA,[ SEÑOR VALDES.],"[[ SEÑORES SILVA señorsa MAGGI, sEÑor], [FUENT...","[[], [ SEÑOR GÓMEZ.]]",2,15,abril,2015-04-02
3,catorce de abril de dos mil quince,PRIMERA,[ SEÑOR SEGURA.],"[[ SEÑORES VALDÉS, SILVA señora MAGGI y EL], []]","[[], [ SEÑOR FIGUEROA.]]",14,15,abril,2015-04-14
4,dieciséis de abril de dos mil quince,PRIMERA,[ SEÑOR VALDÉS.],"[[ SEÑORA MA GGI, señor FUENTES y LOS ABOGADOS...","[[ SEÑORES CORREA y FIGUEROA.], []]",16,15,abril,2015-04-16
...,...,...,...,...,...,...,...,...,...
687,veinticuatro de abril de dos mil quince,CUARTA,[ SEÑOR MUÑOZ.],"[[ SEÑORES DOLMESTCH, CARREÑO, PIERRY,], []]","[[], []]",24,15,abril,2015-04-24
688,diez de abril de dos mil quince,CUARTA,[ SEÑOR MUÑOZ.],"[[ SEÑORES JUICA, DOLMESTCH, VALDES,], []]","[[], []]",10,15,abril,2015-04-10
689,diecisiete de abril de dos mil quince,CUARTA,[ SEÑOR MUÑOZ.],"[[ SEÑORES JUICA, DOLMESTCH, VALDES,], []]","[[], []]",17,15,abril,2015-04-17
690,diecisiete de abril de dos mil quince,CUARTA,[ SEÑOR MUÑOZ.],"[[ SEÑORES JUICA, DOLMESTCH, VALDES,], []]","[[], []]",17,15,abril,2015-04-17


In [787]:
df4['Presidente'] = df4['Presidente'].str[0]

In [789]:

df4['Presidente'] = df4['Presidente'].str.replace('SEÑOR','')
df4['Presidente'] = df4['Presidente'].str.replace('SEÑORA','')
df4['Presidente'] = df4['Presidente'].str.replace('.','')
df4['Presidente'].value_counts()

<ipython-input-789-befdb1825f3a>:3: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will*not* be treated as literal strings when regex=True.
  df4['Presidente'] = df4['Presidente'].str.replace('.','')


  JUICA           163
  BLANCO          132
  MUÑOZ            68
  CARREÑO          61
  PIERR Y          49
  VALDES           34
  SEGURA           28
  VALDÉS           23
  PIERRY           20
  PIERR Y,         16
  VALDEÉS          13
  CISTERNAS        11
  DOLMESTCH        10
  VALDES |          8
  PIERRY,           7
  BLANCO,           6
  KUNSEMULLER,      6
  V ALDES           3
Name: Presidente, dtype: int64

In [790]:
df4['Ministros'] = df3['Ministros'].copy()

In [791]:
df4['Ministros2'] = df4['Ministros'].str[1]
df4['Ministros2'] = df4['Ministros2'].str[0]
df4 = df4.fillna('')
df4['Ministros'] = df4['Ministros'].str[0]
df4['Ministros'] = df4['Ministros'].str[0]

In [427]:
df4['Ministros'] = df4['Ministros'] + ' ' + df4['Ministros2']

In [792]:
df4['Ministros']

0              SEÑORES VALDES, SILVA seÑora MAGGI y
1       SEÑORA MA GGI, señor FUENTES y LOS ABOGADOS
2                SEÑORES SILVA señorsa MAGGI, sEÑor
3           SEÑORES VALDÉS, SILVA señora MAGGI y EL
4       SEÑORA MA GGI, señor FUENTES y LOS ABOGADOS
                           ...                     
687             SEÑORES DOLMESTCH, CARREÑO, PIERRY,
688               SEÑORES JUICA, DOLMESTCH, VALDES,
689               SEÑORES JUICA, DOLMESTCH, VALDES,
690               SEÑORES JUICA, DOLMESTCH, VALDES,
691               SEÑORES JUICA, DOLMESTCH, VALDES,
Name: Ministros, Length: 692, dtype: object

In [793]:
df4['Ministros'] = df4['Ministros'].str.upper()

In [794]:
df4['Ministros'] = df4['Ministros'].str.replace('SEÑORES','')
df4['Ministros'] = df4['Ministros'].str.replace('SEÑORA','')
df4['Ministros'] = df4['Ministros'].str.replace('SEÑOR','')
df4['Ministros'] = df4['Ministros'].str.replace('MA GGI','MAGGI')
df4['Ministros'] = df4['Ministros'].str.replace('MAGGL','MAGGI')
df4['Ministros'] = df4['Ministros'].str.replace('LOS ABOGADOS','')


In [796]:
df4['Ministros'] = df4['Ministros'].str.replace(r'\b\w\b','')

<ipython-input-796-4e758a880aab>:1: FutureWarning: The default value of regex will change from True to False in a future version.
  df4['Ministros'] = df4['Ministros'].str.replace(r'\b\w\b','')


In [797]:

df4['Ministros'] = df4['Ministros'].str.replace(r'\bEL\b','')
df4['Ministros'] = df4['Ministros'].str.replace(r'\bSA\b','')
df4['Ministros'] = df4['Ministros'].str.replace(',','')

<ipython-input-797-1e29ff854d19>:1: FutureWarning: The default value of regex will change from True to False in a future version.
  df4['Ministros'] = df4['Ministros'].str.replace(r'\bEL\b','')
<ipython-input-797-1e29ff854d19>:2: FutureWarning: The default value of regex will change from True to False in a future version.
  df4['Ministros'] = df4['Ministros'].str.replace(r'\bSA\b','')


In [798]:
df4['Ministros']

0            VALDES SILVA  MAGGI 
1                MAGGI  FUENTES  
2                   SILVA  MAGGI 
3           VALDÉS SILVA  MAGGI  
4                MAGGI  FUENTES  
                  ...            
687      DOLMESTCH CARREÑO PIERRY
688        JUICA DOLMESTCH VALDES
689        JUICA DOLMESTCH VALDES
690        JUICA DOLMESTCH VALDES
691        JUICA DOLMESTCH VALDES
Name: Ministros, Length: 692, dtype: object

In [799]:
df4['Integrantes2'] = df4['Integrantes'].str[1]
df4['Integrantes2'] = df4['Integrantes2'].str[0]

In [800]:
df4['Integrantes'] = df4['Integrantes'].str[0]
df4['Integrantes'] = df4['Integrantes'].str[0]

In [801]:
df4= df4.fillna('')

In [802]:
df4['Integrantes'] = df4['Integrantes'] + ' ' + df4['Integrantes2']

In [803]:
df4['Integrantes']

0                                  
1       SEÑORES CORREA y FIGUEROA. 
2                      SEÑOR GÓMEZ.
3                   SEÑOR FIGUEROA.
4       SEÑORES CORREA y FIGUEROA. 
                   ...             
687                                
688                                
689                                
690                                
691                                
Name: Integrantes, Length: 692, dtype: object

In [804]:
df4['Integrantes'] = df4['Integrantes'].str.upper()

In [805]:
df4['Integrantes'] =  df4['Integrantes'].str.replace('SEÑORES','')
df4['Integrantes'] = df4['Integrantes'].str.replace('SEÑORA','')
df4['Integrantes'] = df4['Integrantes'].str.replace('SEÑOR','')
df4['Integrantes'] = df4['Integrantes'].str.replace('.','')
df4['Integrantes'] = df4['Integrantes'].str.replace(r'\b\w\b','')


<ipython-input-805-9ea5d7be16cf>:4: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will*not* be treated as literal strings when regex=True.
  df4['Integrantes'] = df4['Integrantes'].str.replace('.','')
<ipython-input-805-9ea5d7be16cf>:5: FutureWarning: The default value of regex will change from True to False in a future version.
  df4['Integrantes'] = df4['Integrantes'].str.replace(r'\b\w\b','')


In [806]:
df4['Abogados'] = df4['Integrantes'].copy()

In [807]:
df4['Integrantes'] = df4['Ministros'] + '' + df4['Abogados']

In [808]:
df4['Integrantes']

0                      VALDES SILVA  MAGGI  
1        MAGGI  FUENTES    CORREA  FIGUEROA 
2                      SILVA  MAGGI    GÓMEZ
3           VALDÉS SILVA  MAGGI     FIGUEROA
4        MAGGI  FUENTES    CORREA  FIGUEROA 
                       ...                  
687                DOLMESTCH CARREÑO PIERRY 
688                  JUICA DOLMESTCH VALDES 
689                  JUICA DOLMESTCH VALDES 
690                  JUICA DOLMESTCH VALDES 
691                  JUICA DOLMESTCH VALDES 
Name: Integrantes, Length: 692, dtype: object

In [809]:
df4['Integrantes'] = df4['Integrantes'].str.replace('   ',' ')
df4['Integrantes'] = df4['Integrantes'].str.replace('  ',' ')

In [810]:
df4['Integrantes'] = df4['Integrantes'].str.lstrip()

In [811]:
df4['Integrantes2'] = df4['Integrantes'].str.replace(" ", ",")


In [812]:
df4['Integrantes2'] = df4["Integrantes2"].str.split(',', n = 3, expand = False)

In [813]:
df4['Integrante1'] = df4['Integrantes2'].str[0]
df4['Integrante2'] = df4['Integrantes2'].str[1]
df4['Integrante3'] = df4['Integrantes2'].str[2]
df4['Integrante3'] = df4['Integrantes2'].str[4]

In [815]:
len(df4)

,Fecha,Sala,Presidente,Ministros,Integrantes,Dia,Ano,Mes,Fecha2,Ministros2,Integrantes2,Abogados,Integrante1,Integrante2,Integrante3
0,trece de abril de dos mil quince,PRIMERA,SEGURA,VALDES SILVA MAGGI,VALDES SILVA MAGGI,13,15,abril,2015-04-13 00:00:00,,"[VALDES, SILVA, MAGGI, ]",,VALDES,SILVA,NaN
1,dieciséis de abril de dos mil quince,PRIMERA,VALDÉS,MAGGI FUENTES,MAGGI FUENTES CORREA FIGUEROA,16,15,abril,2015-04-16 00:00:00,,"[MAGGI, FUENTES, CORREA, FIGUEROA,]",CORREA FIGUEROA,MAGGI,FUENTES,NaN
2,dos de abril de dos mil quince,PRIMERA,VALDES,SILVA MAGGI,SILVA MAGGI GÓMEZ,2,15,abril,2015-04-02 00:00:00,FUENTES,"[SILVA, MAGGI, GÓMEZ]",GÓMEZ,SILVA,MAGGI,NaN
3,catorce de abril de dos mil quince,PRIMERA,SEGURA,VALDÉS SILVA MAGGI,VALDÉS SILVA MAGGI FIGUEROA,14,15,abril,2015-04-14 00:00:00,,"[VALDÉS, SILVA, MAGGI, ,FIGUEROA]",FIGUEROA,VALDÉS,SILVA,NaN
4,dieciséis de abril de dos mil quince,PRIMERA,VALDÉS,MAGGI FUENTES,MAGGI FUENTES CORREA FIGUEROA,16,15,abril,2015-04-16 00:00:00,,"[MAGGI, FUENTES, CORREA, FIGUEROA,]",CORREA FIGUEROA,MAGGI,FUENTES,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
687,veinticuatro de abril de dos mil quince,CUARTA,MUÑOZ,DOLMESTCH CARREÑO PIERRY,DOLMESTCH CARREÑO PIERRY,24,15,abril,2015-04-24 00:00:00,,"[DOLMESTCH, CARREÑO, PIERRY, ]",,DOLMESTCH,CARREÑO,NaN
688,diez de abril de dos mil quince,CUARTA,MUÑOZ,JUICA DOLMESTCH VALDES,JUICA DOLMESTCH VALDES,10,15,abril,2015-04-10 00:00:00,,"[JUICA, DOLMESTCH, VALDES, ]",,JUICA,DOLMESTCH,NaN
689,diecisiete de abril de dos mil quince,CUARTA,MUÑOZ,JUICA DOLMESTCH VALDES,JUICA DOLMESTCH VALDES,17,15,abril,2015-04-17 00:00:00,,"[JUICA, DOLMESTCH, VALDES, ]",,JUICA,DOLMESTCH,NaN
690,diecisiete de abril de dos mil quince,CUARTA,MUÑOZ,JUICA DOLMESTCH VALDES,JUICA DOLMESTCH VALDES,17,15,abril,2015-04-17 00:00:00,,"[JUICA, DOLMESTCH, VALDES, ]",,JUICA,DOLMESTCH,NaN


In [816]:
df5 = df4
df5 = df5.drop(['Fecha', 'Integrantes2', 'Dia', 'Ano', 'Mes'], axis = 1)


In [818]:
len(df5)

692

In [819]:
df5 = df5.drop_duplicates()
len(df5)

64

In [820]:
df5.to_csv("Primer intento scan")